# Generation of training data for thermal air conditioning control

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import datetime
import time

current_dir

'C:\\Users\\juliu\\Desktop\\pmv-air-conditioning-control\\code\\src'

In [32]:
filename = "output_PMV_PPD_data.csv"
filepath = "./"
export_filepath = "./output/"
logpath = "./logs/"
data = pd.read_csv(filepath + filename)

In [11]:
datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

'2021-11-16_18:12:54'

In [13]:
def optimal_pmv(data, index, pmv_val):
    """
    PARAMETERS:
    data:			data frame of calculated PMV values by Fanger equations
    index:			current index of dataframe
    pmv_val:		current pmv_value at respective index
    
    WHEN  -0.5 > PMV > 0.5 : ventilation OFF, heating OFF, cooling OFF
    """
    # if-statement is redundant here but kept for security reasons
    if (pmv_val > -0.5) & (pmv_val < 0.5):

        # ventilation OFF
        data.loc[index, 'PMV_vent'] = pmv_val
        data.loc[index, 'd_PMV_vent'] = 0                    
        data.loc[index, 'vent_lvl'] = 0

        # heating OFF
        data.loc[index, 'PMV_heat'] = pmv_val
        data.loc[index, 'd_PMV_heat'] = 0
        data.loc[index, 'heat_lvl'] = 0                                    

        # cooling OFF
        data.loc[index, 'PMV_cool'] = pmv_val
        data.loc[index, 'd_PMV_cool'] = 0
        data.loc[index, 'cool_lvl'] = 0                    
        
        return True, pmv_val
    else:
        return False, pmv_val

In [14]:
def heating(data, index, pmv_val, heatval, digits_heat, tmp, hum, vel):
    """
    Calculation of heating levels and respective PMV heating impact.
    
    PARAMETERS:
    data:			data frame of calculated PMV values by Fanger equations
    index:			current index of dataframe
    pmv_val:		current pmv_value at respective index
    heatval:		array of values for the heat conditions (d/dT in respect of numbers of people in one room)
    digits_heat:	number of digits to round the value
    tmp:			current temperature value used in loop
    hum:			current humidity value used in loop
    vel:			current velocity value used in loop
    """
    
    # Energy levels in KW
    energy_lvl1 = 1.3
    energy_lvl2 = 2.2
    energy_lvl3 = 3.1
    energy_lvl4 = 4.0
    energy_lvl5 = 4.9
    
    # WHEN PMV < -0.5 : ventilation OFF, heating ON, cooling OFF    
    # if-statement is redundant here but kept for security reasons
    if (pmv_val <= -0.5):

        # ventilation OFF
        data.loc[index, 'PMV_vent'] = data.loc[index, 'PMV']
        data.loc[index, 'd_PMV_vent'] = 0
        data.loc[index, 'vent_lvl'] = 0

        # cooling OFF
        data.loc[index, 'PMV_cool'] = data.loc[index, 'PMV_vent']
        data.loc[index, 'd_PMV_cool'] = 0
        data.loc[index, 'cool_lvl'] = 0

        # define heating level conditions (PMV value for comparison below)
        heat_condition_1 = data[((data['Temperature'] == round(tmp + heatval[0], digits_heat)) & 
                   (data['Humidity'] == hum) & 
                   (data['Air_velocity'] == vel))].PMV

        heat_condition_2 = data[((data['Temperature'] == round(tmp + heatval[1], digits_heat)) & 
                   (data['Humidity'] == hum) & 
                   (data['Air_velocity'] == vel))].PMV

        heat_condition_3 = data[((data['Temperature'] == round(tmp + heatval[2], digits_heat)) & 
                   (data['Humidity'] == hum) & 
                   (data['Air_velocity'] == vel))].PMV

        heat_condition_4 = data[((data['Temperature'] == round(tmp + heatval[3], digits_heat)) & 
                   (data['Humidity'] == hum) & 
                   (data['Air_velocity'] == vel))].PMV        

        heat_condition_5 = data[((data['Temperature'] == round(tmp + heatval[4], digits_heat)) & 
                   (data['Humidity'] == hum) & 
                   (data['Air_velocity'] == vel))].PMV


        if (float(heat_condition_1) >= -0.5):
            data.loc[index, 'heat_lvl'] = 1
            data.loc[index, 'PMV_heat'] = float(heat_condition_1)
            data.loc[index, 'd_PMV_heat'] = float(heat_condition_1) - pmv_val
            data.loc[index, 'heat_KW'] = energy_lvl1
            
        
        elif (float(heat_condition_2) >= -0.5):
            data.loc[index, 'heat_lvl'] = 2
            data.loc[index, 'PMV_heat'] = float(heat_condition_2)
            data.loc[index, 'd_PMV_heat'] = float(heat_condition_2) - pmv_val
            data.loc[index, 'heat_KW'] = energy_lvl2
         
        elif (float(heat_condition_3) >= -0.5):
            data.loc[index, 'heat_lvl'] = 3
            data.loc[index, 'PMV_heat'] = float(heat_condition_3)
            data.loc[index, 'd_PMV_heat'] = float(heat_condition_3) - pmv_val
            data.loc[index, 'heat_KW'] = energy_lvl3
        
        elif (float(heat_condition_4) >= -0.5):
            data.loc[index, 'heat_lvl'] = 4
            data.loc[index, 'PMV_heat'] = float(heat_condition_4)
            data.loc[index, 'd_PMV_heat'] = float(heat_condition_4) - pmv_val
            data.loc[index, 'heat_KW'] = energy_lvl4
        
        else:
        #elif (float(heat_condition_3) > -0.5):
            data.loc[index, 'heat_lvl'] = 5
            data.loc[index, 'PMV_heat'] = float(heat_condition_5)
            data.loc[index, 'd_PMV_heat'] = float(heat_condition_5) - pmv_val
            data.loc[index, 'heat_KW'] = energy_lvl5
        
        return True, float(data.loc[index, 'PMV_heat'])
    else:
        return False, pmv_val

In [15]:
def ventilation(data, index, pmv_val, ventval, digits_vent, tmp, hum, vel):
    """
    Calculation of ventilation levels and respective PMV ventilation impact.
    
    PARAMETERS:
    data:			data frame of calculated PMV values by Fanger equations
    index:			current index of dataframe
    pmv_val:		current pmv_value at respective index
    vent_val:		array of values for the heat ventilation (d/dT in respect of numbers of people in one room)
    digits_vent:	number of digits to round the value
    tmp:			current temperature value used in loop
    hum:			current humidity value used in loop
    vel:			current velocity value used in loop
    """
    # Energy levels in KW
    energy_lvl1 = 0.1
    energy_lvl2 = 0.2
    energy_lvl3 = 0.3
    
    # WHEN PMV > 0.5 : ventilation ON(check), heating OFF, cooling ON(check)
    if (pmv_val >= 0.5):                

        # define ventilation level conditions (PMV value for comparison below)
        vent_condition_1 = data[((data['Temperature'] == tmp) & 
                   (data['Humidity'] == hum) & 
                   (data['Air_velocity'] == round(vel + ventval[0], digits_vent)))].PMV

        vent_condition_2 = data[((data['Temperature'] == tmp) & 
                   (data['Humidity'] == hum) & 
                   (data['Air_velocity'] == round(vel + ventval[1], digits_vent)))].PMV

        vent_condition_3 = data[((data['Temperature'] == tmp) & 
                   (data['Humidity'] == hum) & 
                   (data['Air_velocity'] == round(vel + ventval[2], digits_vent)))].PMV


        # check if vent_condition_1 exceeds the maximum room air velocity threshhold (0.7)
        # (threshhold can be changed by calculation new PMV Fanger data with other resolution and max value)
        if (vent_condition_1.empty):
            logger.info('\nIndex: \t\t\t' + str(index.values) + '\nVentilation condition 1 is empty\n')
            # print('\n vent con empty check 1\n')
            # print(index)
            data.loc[index, 'vent_lvl'] = 0
            data.loc[index, 'PMV_vent'] = data.loc[index, 'PMV']
            data.loc[index, 'd_PMV_vent'] = 0                        
            return True, float(data.loc[index, 'PMV_vent'])

        # check if vent level 1 is sufficient to regulate the PMV to <= 0.5
        elif (float(vent_condition_1) <= 0.5):
            data.loc[index, 'vent_lvl'] = 1
            data.loc[index, 'PMV_vent'] = float(vent_condition_1)
            data.loc[index, 'd_PMV_vent'] = float(vent_condition_1) - pmv_val
            data.loc[index, 'vent_KW'] = energy_lvl1

        # check if vent_condition_2 exceeds the maximum room air velocity threshhold 
        if (vent_condition_2.empty):
            logger.info('\nIndex: \t\t\t' + str(index.values) + '\nVentilation condition 2 is empty\n')
            # print('\n vent con empty check 2\n')
            # print(index)
            data.loc[index, 'vent_lvl'] = 1
            data.loc[index, 'PMV_vent'] = float(vent_condition_1)
            data.loc[index, 'd_PMV_vent'] = float(vent_condition_1) - pmv_val
            data.loc[index, 'vent_KW'] = energy_lvl1
            return True, float(vent_condition_1)

        # check if vent level 2 is sufficient to regulate the PMV to <= 0.5                        
        elif (float(vent_condition_2) <= 0.5):
            data.loc[index, 'vent_lvl'] = 2
            data.loc[index, 'PMV_vent'] = float(vent_condition_2)
            data.loc[index, 'd_PMV_vent'] = float(vent_condition_2) - pmv_val
            data.loc[index, 'vent_KW'] = energy_lvl2

        # check if vent_condition_3 exceeds the maximum room air velocity threshhold 
        if (vent_condition_3.empty):
            logger.info('\nIndex: \t\t\t' + str(index.values) + '\nVentilation condition 3 is empty\n')
            # print('\n vent con empty check 3\n')
            # print(index)
            data.loc[index, 'vent_lvl'] = 2
            data.loc[index, 'PMV_vent'] = float(vent_condition_2)
            data.loc[index, 'd_PMV_vent'] = float(vent_condition_2) - pmv_val
            data.loc[index, 'vent_KW'] = energy_lvl2
            return True, float(vent_condition_2)

        # activate vent level 3                                        
        else:
            data.loc[index, 'vent_lvl'] = 3
            data.loc[index, 'PMV_vent'] = float(vent_condition_3)
            data.loc[index, 'd_PMV_vent'] = float(vent_condition_3) - pmv_val
            data.loc[index, 'vent_KW'] = energy_lvl3
            
        # heating OFF
        data.loc[index, 'PMV_heat'] = data.loc[index, 'PMV_vent']
        data.loc[index, 'd_PMV_heat'] = 0
        data.loc[index, 'heat_lvl'] = 0
        
        return True, float(data.loc[index, 'PMV_vent'])
    else:
        return False, pmv_val

In [16]:
def cooling(data, index, pmv_val, coolval, digits_cool, tmp, hum, vel):
    """
    Calculation of cooling levels and respective PMV cooling impact.
    
    PARAMETERS:
    data:			data frame of calculated PMV values by Fanger equations
    index:			current index of dataframe
    pmv_val:		current pmv_value at respective index
    heatval:		array of values for the heat conditions (d/dT in respect of numbers of people in one room)
    digits_heat:	number of digits to round the value
    tmp:			current temperature value used in loop
    hum:			current humidity value used in loop
    vel:			current velocity value used in loop
    """ 
    
    # Energy levels in KW
    energy_lvl1 = 0.90
    energy_lvl2 = 2.45
    energy_lvl3 = 4.00
    
    
    # WHEN PMV >= -0.5 : ventilation ON/OFF, heating OFF, cooling ON
    if (pmv_val>=0.5):
        # cooling ON
        # define cooling level conditions (PMV value for comparison below)
        cool_condition_1 = data[((data['Temperature'] == round(tmp + coolval[0], digits_cool)) & 
                   (data['Humidity'] == hum) & 
                   (data['Air_velocity'] == vel))].PMV

        cool_condition_2 = data[((data['Temperature'] == round(tmp + coolval[1], digits_cool)) & 
                   (data['Humidity'] == hum) & 
                   (data['Air_velocity'] == vel))].PMV

        cool_condition_3 = data[((data['Temperature'] == round(tmp + coolval[2], digits_cool)) & 
                   (data['Humidity'] == hum) & 
                   (data['Air_velocity'] == vel))].PMV    

        if (float(cool_condition_1) <= 0.5):
            # cooling lvl 1 ON
            data.loc[index, 'cool_lvl'] = 1
            data.loc[index, 'PMV_cool'] = float(cool_condition_1)
            data.loc[index, 'd_PMV_cool'] = float(cool_condition_1) - pmv_val
            data.loc[index, 'cool_KW'] = energy_lvl1
            
            # heating OFF
            data.loc[index, 'PMV_heat'] = data.loc[index, 'PMV_vent']
            data.loc[index, 'd_PMV_heat'] = 0
            data.loc[index, 'heat_lvl'] = 0 
             
        elif (float(cool_condition_2) <= 0.5):
            # cooling lvl 2 ON
            data.loc[index, 'cool_lvl'] = 2
            data.loc[index, 'PMV_cool'] = float(cool_condition_2)
            data.loc[index, 'd_PMV_cool'] = float(cool_condition_2) - pmv_val
            data.loc[index, 'cool_KW'] = energy_lvl2
            
            # heating OFF
            data.loc[index, 'PMV_heat'] = data.loc[index, 'PMV_vent']
            data.loc[index, 'd_PMV_heat'] = 0
            data.loc[index, 'heat_lvl'] = 0 
        
        # elif (float(cool_condition_3) <= 0.5):
        else:
            # cooling lvl 3 ON
            data.loc[index, 'cool_lvl'] = 3
            data.loc[index, 'PMV_cool'] = float(cool_condition_3)
            data.loc[index, 'd_PMV_cool'] = float(cool_condition_3) - pmv_val
            data.loc[index, 'cool_KW'] = energy_lvl3
            
            # heating OFF
            data.loc[index, 'PMV_heat'] = data.loc[index, 'PMV_vent']
            data.loc[index, 'd_PMV_heat'] = 0
            data.loc[index, 'heat_lvl'] = 0 

        return True, float(data.loc[index, 'PMV_cool'])
    
    else:
        # cooling OFF
        data.loc[index, 'PMV_cool'] = data.loc[index, 'PMV_vent']
        data.loc[index, 'd_PMV_cool'] = 0
        data.loc[index, 'cool_lvl'] = 0
        
        # heating OFF
        data.loc[index, 'PMV_heat'] = data.loc[index, 'PMV_vent']
        data.loc[index, 'd_PMV_heat'] = 0
        data.loc[index, 'heat_lvl'] = 0 
        
        return False, pmv_val



In [17]:
def calculate_final_PMV(data, index, pmv_val, coolval, digits_cool, tmp, hum, vel):
    """
    Function summarizes the dPMV values of the functions "heating", "cooling", "ventilation" 
    and calculates final values for dPMV and PMV.
    
    """
    d_pmv_final = data.loc[index, 'd_PMV_heat'] + data.loc[index, 'd_PMV_vent'] + data.loc[index, 'd_PMV_cool']
    pmv_final = data.loc[index, 'PMV'] + d_pmv_final
    
    data.loc[index, 'PMV_final'] = pmv_final
    data.loc[index, 'd_PMV_final'] = d_pmv_final
    
    if (float(pmv_final) >= -0.5) & (float(pmv_final) <= 0.5):
        data.loc[index, 'success'] = True
    else:
        data.loc[index, 'success'] = False

In [18]:
def calculate_final_PPD(data, index):
    """
    Calculates PPD value based on respective PMV value.
    """
    pmv_final = data.loc[index, 'PMV_final']
    ppd_final = 100 - 95 * math.exp(-0.03353 * pow(pmv_final, 4) - 0.2179 * pow(pmv_final, 2))
    data.loc[index, 'PPD_final'] = ppd_final
    return True

In [19]:
def calculate_final_KW(data, index):
    """
    Calculates final KW value based.
    """
    heat_kw = data.loc[index, 'heat_KW']
    cool_kw = data.loc[index, 'cool_KW'] 
    vent_kw = data.loc[index, 'vent_KW']
    kw_final = np.nansum(np.array([float(heat_kw), float(cool_kw), float(vent_kw)]))
    data.loc[index, 'KW_final'] = kw_final
    return True

In [20]:
data.drop(columns=['PPD', 'met', 'clo'], inplace=True)

data.round(2)

,entry_id,Temperature,Humidity,Air_velocity,PMV
0,1.0,16.0,30.0,0.00,-2.12
1,2.0,16.0,30.0,0.05,-2.12
2,3.0,16.0,30.0,0.10,-2.12
3,4.0,16.0,30.0,0.15,-2.28
4,5.0,16.0,30.0,0.20,-2.43
...,...,...,...,...,...
107860,107861.0,30.0,80.0,0.50,1.97
107861,107862.0,30.0,80.0,0.55,1.96
107862,107863.0,30.0,80.0,0.60,1.95
107863,107864.0,30.0,80.0,0.65,1.94


In [21]:
# add necessary columns to dataframe

data['d_PMV_vent'] = pd.Series()
data['PMV_vent'] = pd.Series()
data['vent_lvl'] = pd.Series()
data['vent_KW'] = pd.Series()

data['d_PMV_cool'] = pd.Series()
data['PMV_cool'] = pd.Series()
data['cool_lvl'] = pd.Series()
data['cool_KW'] = pd.Series()


data['d_PMV_heat'] = pd.Series()
data['PMV_heat'] = pd.Series()
data['heat_lvl'] = pd.Series()
data['heat_KW'] = pd.Series()

data['d_PMV_final'] = pd.Series()
data['PMV_final'] = pd.Series()
data['PPD_final'] = pd.Series()
data['KW_final'] = pd.Series()
data['success'] = pd.Series()

data

,entry_id,Temperature,Humidity,Air_velocity,PMV,d_PMV_vent,PMV_vent,vent_lvl,vent_KW,d_PMV_cool,...,cool_KW,d_PMV_heat,PMV_heat,heat_lvl,heat_KW,d_PMV_final,PMV_final,PPD_final,KW_final,success
0,1.0,16.0,30.0,0.00,-2.12,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,16.0,30.0,0.05,-2.12,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,16.0,30.0,0.10,-2.12,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,16.0,30.0,0.15,-2.28,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,16.0,30.0,0.20,-2.43,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107860,107861.0,30.0,80.0,0.50,1.97,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107861,107862.0,30.0,80.0,0.55,1.96,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107862,107863.0,30.0,80.0,0.60,1.95,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107863,107864.0,30.0,80.0,0.65,1.94,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# HEATING VALUES
digits_heat = 1

heatval1 = round(2.30, digits_heat)
heatval2 = round(3.89, digits_heat)
heatval3 = round(5.49, digits_heat)
heatval4 = round(7.08, digits_heat)
heatval5 = round(8.67, digits_heat)
heatval = [heatval1, heatval2, heatval3, heatval4, heatval5]

# VENTILATION VALUES
digits_vent = 2

ventval1 = 0.15 
ventval2 = 0.30
ventval3 = 0.45
ventval = [ventval1, ventval2, ventval3]

# COOLING VALUES
digits_cool = 1

coolval1 = round(-1.59, digits_heat)
coolval2 = round(-4.34, digits_heat)
coolval3 = round(-7.08, digits_heat)
coolval = [coolval1, coolval2, coolval3]

# FLAGS (INITIALISATION)
optimal_flag = False
heating_flag = False
ventilation_flag = False
cooling_flag = False

In [23]:
start_time = time.time()

# MAIN 

temperatures = np.unique(data['Temperature'].to_numpy())
humidities = np.unique(data['Humidity'].to_numpy())
velocities = np.unique(data['Air_velocity'].to_numpy())

for tmp in temperatures:
        for hum in humidities:
            for vel in velocities:

                # save index
                index = data[(data['Temperature'] == tmp) & 
                         (data['Humidity'] == hum) & 
                         (data['Air_velocity'] == vel)].index 
                
                # save PMV value
                pmv_val = float(data[(data['Temperature'] == tmp) & 
                               (data['Humidity'] == hum) & 
                               (data['Air_velocity'] == vel)].PMV)
                
                
                #print('\nIteration: ' + str(index.values))
                
                if (pmv_val > -0.5) & (pmv_val < 0.5):
                    # function call
                    optimal_flag, pmv_val = optimal_pmv(data, index, pmv_val)  
                    # print('optimal flag: ' + str(optimal_flag))
                    #logger.info('\nIndex: \t\t\t' + str(index) + '\nOptimal Flag: \t\t'+ str(optimal_flag)+'\n')
                    
                elif (pmv_val <= -0.5):
                    # function call
                    heating_flag, pmv_val = heating(data, index, pmv_val, heatval, digits_heat, tmp, hum, vel)
                    # print('heating flag: ' + str(heating_flag))
                    #logger.info('\nIndex: \t\t\t' + str(index) + '\nHeating Flag: \t\t'+ str(heating_flag)+'\n')

                
                elif (pmv_val >= 0.5):
                    # function call
                    ventilation_flag, pmv_val = ventilation(data, index, pmv_val, ventval, digits_vent, tmp, hum, vel)
                    # print('ventilation flag: ' + str(ventilation_flag))
                    #logger.info('\nIndex: \t\t' + str(index) + '\nVentilation Flag: \t'+ str(ventilation_flag)+'\n')
                    cooling_flag, pmv_val = cooling(data, index, pmv_val, coolval, digits_cool, tmp, hum, vel)
                    # print('cooling flag: ' + str(cooling_flag))
                    
                logger.info('\nIndex: \t\t\t' + str(index.values) + 
                            '\nOptimal Flag: \t\t'+ str(optimal_flag) + 
                            '\nHeating Flag: \t\t'+ str(heating_flag)+
                            '\nVentilation Flag: \t'+ str(ventilation_flag)+
                            '\nCooling Flag: \t\t'+ str(cooling_flag)+'\n')

                calculate_final_PMV(data, index, pmv_val, coolval, digits_cool, tmp, hum, vel)
                calculate_final_PPD(data, index)
                calculate_final_KW(data, index)
                
                if not(index.values % 100):
                    print(index)

                
                
            
# data.to_csv(str(datetime.date.today()) + '_DEBUG_refactor_functions.csv')                 

stop_time = time.time()

time = stop_time-start_time

print('Calculation time in secounds: ' + str(time))

logger.info('\nCalculation time in secounds: ' + str(time))

data.to_csv(export_filepath + str(datetime.date.today()) + '_training_data.csv')

Int64Index([0], dtype='int64')
Int64Index([100], dtype='int64')
Int64Index([200], dtype='int64')
Int64Index([300], dtype='int64')
Int64Index([400], dtype='int64')
Int64Index([500], dtype='int64')
Int64Index([600], dtype='int64')
Int64Index([700], dtype='int64')
Int64Index([800], dtype='int64')
Int64Index([900], dtype='int64')
Int64Index([1000], dtype='int64')
Int64Index([1100], dtype='int64')
Int64Index([1200], dtype='int64')
Int64Index([1300], dtype='int64')
Int64Index([1400], dtype='int64')
Int64Index([1500], dtype='int64')
Int64Index([1600], dtype='int64')
Int64Index([1700], dtype='int64')
Int64Index([1800], dtype='int64')
Int64Index([1900], dtype='int64')
Int64Index([2000], dtype='int64')
Int64Index([2100], dtype='int64')
Int64Index([2200], dtype='int64')
Int64Index([2300], dtype='int64')
Int64Index([2400], dtype='int64')
Int64Index([2500], dtype='int64')
Int64Index([2600], dtype='int64')
Int64Index([2700], dtype='int64')
Int64Index([2800], dtype='int64')
Int64Index([2900], dtype='

Int64Index([23800], dtype='int64')
Int64Index([23900], dtype='int64')
Int64Index([24000], dtype='int64')
Int64Index([24100], dtype='int64')
Int64Index([24200], dtype='int64')
Int64Index([24300], dtype='int64')
Int64Index([24400], dtype='int64')
Int64Index([24500], dtype='int64')
Int64Index([24600], dtype='int64')
Int64Index([24700], dtype='int64')
Int64Index([24800], dtype='int64')
Int64Index([24900], dtype='int64')
Int64Index([25000], dtype='int64')
Int64Index([25100], dtype='int64')
Int64Index([25200], dtype='int64')
Int64Index([25300], dtype='int64')
Int64Index([25400], dtype='int64')
Int64Index([25500], dtype='int64')
Int64Index([25600], dtype='int64')
Int64Index([25700], dtype='int64')
Int64Index([25800], dtype='int64')
Int64Index([25900], dtype='int64')
Int64Index([26000], dtype='int64')
Int64Index([26100], dtype='int64')
Int64Index([26200], dtype='int64')
Int64Index([26300], dtype='int64')
Int64Index([26400], dtype='int64')
Int64Index([26500], dtype='int64')
Int64Index([26600], 

Int64Index([47300], dtype='int64')
Int64Index([47400], dtype='int64')
Int64Index([47500], dtype='int64')
Int64Index([47600], dtype='int64')
Int64Index([47700], dtype='int64')
Int64Index([47800], dtype='int64')
Int64Index([47900], dtype='int64')
Int64Index([48000], dtype='int64')
Int64Index([48100], dtype='int64')
Int64Index([48200], dtype='int64')
Int64Index([48300], dtype='int64')
Int64Index([48400], dtype='int64')
Int64Index([48500], dtype='int64')
Int64Index([48600], dtype='int64')
Int64Index([48700], dtype='int64')
Int64Index([48800], dtype='int64')
Int64Index([48900], dtype='int64')
Int64Index([49000], dtype='int64')
Int64Index([49100], dtype='int64')
Int64Index([49200], dtype='int64')
Int64Index([49300], dtype='int64')
Int64Index([49400], dtype='int64')
Int64Index([49500], dtype='int64')
Int64Index([49600], dtype='int64')
Int64Index([49700], dtype='int64')
Int64Index([49800], dtype='int64')
Int64Index([49900], dtype='int64')
Int64Index([50000], dtype='int64')
Int64Index([50100], 

Int64Index([70800], dtype='int64')
Int64Index([70900], dtype='int64')
Int64Index([71000], dtype='int64')
Int64Index([71100], dtype='int64')
Int64Index([71200], dtype='int64')
Int64Index([71300], dtype='int64')
Int64Index([71400], dtype='int64')
Int64Index([71500], dtype='int64')
Int64Index([71600], dtype='int64')
Int64Index([71700], dtype='int64')
Int64Index([71800], dtype='int64')
Int64Index([71900], dtype='int64')
Int64Index([72000], dtype='int64')
Int64Index([72100], dtype='int64')
Int64Index([72200], dtype='int64')
Int64Index([72300], dtype='int64')
Int64Index([72400], dtype='int64')
Int64Index([72500], dtype='int64')
Int64Index([72600], dtype='int64')
Int64Index([72700], dtype='int64')
Int64Index([72800], dtype='int64')
Int64Index([72900], dtype='int64')
Int64Index([73000], dtype='int64')
Int64Index([73100], dtype='int64')
Int64Index([73200], dtype='int64')
Int64Index([73300], dtype='int64')
Int64Index([73400], dtype='int64')
Int64Index([73500], dtype='int64')
Int64Index([73600], 

Int64Index([94300], dtype='int64')
Int64Index([94400], dtype='int64')
Int64Index([94500], dtype='int64')
Int64Index([94600], dtype='int64')
Int64Index([94700], dtype='int64')
Int64Index([94800], dtype='int64')
Int64Index([94900], dtype='int64')
Int64Index([95000], dtype='int64')
Int64Index([95100], dtype='int64')
Int64Index([95200], dtype='int64')
Int64Index([95300], dtype='int64')
Int64Index([95400], dtype='int64')
Int64Index([95500], dtype='int64')
Int64Index([95600], dtype='int64')
Int64Index([95700], dtype='int64')
Int64Index([95800], dtype='int64')
Int64Index([95900], dtype='int64')
Int64Index([96000], dtype='int64')
Int64Index([96100], dtype='int64')
Int64Index([96200], dtype='int64')
Int64Index([96300], dtype='int64')
Int64Index([96400], dtype='int64')
Int64Index([96500], dtype='int64')
Int64Index([96600], dtype='int64')
Int64Index([96700], dtype='int64')
Int64Index([96800], dtype='int64')
Int64Index([96900], dtype='int64')
Int64Index([97000], dtype='int64')
Int64Index([97100], 

In [25]:
data.round(2)

,entry_id,Temperature,Humidity,Air_velocity,PMV,d_PMV_vent,PMV_vent,vent_lvl,vent_KW,d_PMV_cool,...,cool_KW,d_PMV_heat,PMV_heat,heat_lvl,heat_KW,d_PMV_final,PMV_final,PPD_final,KW_final,success
0,1.0,16.0,30.0,0.00,-2.12,0.00,-2.12,0.0,NaN,0.00,...,NaN,1.90,-0.22,4.0,4.0,1.90,-0.22,6.00,4.00,True
1,2.0,16.0,30.0,0.05,-2.12,0.00,-2.12,0.0,NaN,0.00,...,NaN,1.90,-0.22,4.0,4.0,1.90,-0.22,6.00,4.00,True
2,3.0,16.0,30.0,0.10,-2.12,0.00,-2.12,0.0,NaN,0.00,...,NaN,1.85,-0.27,4.0,4.0,1.85,-0.27,6.51,4.00,True
3,4.0,16.0,30.0,0.15,-2.28,0.00,-2.28,0.0,NaN,0.00,...,NaN,1.90,-0.38,4.0,4.0,1.90,-0.38,8.01,4.00,True
4,5.0,16.0,30.0,0.20,-2.43,0.00,-2.43,0.0,NaN,0.00,...,NaN,1.97,-0.46,4.0,4.0,1.97,-0.46,9.42,4.00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107860,107861.0,30.0,80.0,0.50,1.97,-0.03,1.94,1.0,0.1,-1.45,...,2.45,0.00,1.94,0.0,NaN,-1.48,0.49,10.02,2.55,True
107861,107862.0,30.0,80.0,0.55,1.96,-0.03,1.93,1.0,0.1,-1.47,...,2.45,0.00,1.93,0.0,NaN,-1.50,0.46,9.42,2.55,True
107862,107863.0,30.0,80.0,0.60,1.95,0.00,1.95,0.0,NaN,-1.51,...,2.45,0.00,1.95,0.0,NaN,-1.51,0.44,9.04,2.45,True
107863,107864.0,30.0,80.0,0.65,1.94,0.00,1.94,0.0,NaN,-1.52,...,2.45,0.00,1.94,0.0,NaN,-1.52,0.42,8.68,2.45,True


In [36]:
data.round(2).to_csv(export_filepath + '_output_training_data.csv')

In [27]:
data_filtered = data.copy()
data_filtered

,entry_id,Temperature,Humidity,Air_velocity,PMV,d_PMV_vent,PMV_vent,vent_lvl,vent_KW,d_PMV_cool,...,cool_KW,d_PMV_heat,PMV_heat,heat_lvl,heat_KW,d_PMV_final,PMV_final,PPD_final,KW_final,success
0,1.0,16.0,30.0,0.00,-2.12,0.00,-2.12,0.0,NaN,0.00,...,NaN,1.90,-0.22,4.0,4.0,1.90,-0.22,6.004023,4.00,True
1,2.0,16.0,30.0,0.05,-2.12,0.00,-2.12,0.0,NaN,0.00,...,NaN,1.90,-0.22,4.0,4.0,1.90,-0.22,6.004023,4.00,True
2,3.0,16.0,30.0,0.10,-2.12,0.00,-2.12,0.0,NaN,0.00,...,NaN,1.85,-0.27,4.0,4.0,1.85,-0.27,6.513804,4.00,True
3,4.0,16.0,30.0,0.15,-2.28,0.00,-2.28,0.0,NaN,0.00,...,NaN,1.90,-0.38,4.0,4.0,1.90,-0.38,8.006954,4.00,True
4,5.0,16.0,30.0,0.20,-2.43,0.00,-2.43,0.0,NaN,0.00,...,NaN,1.97,-0.46,4.0,4.0,1.97,-0.46,9.416873,4.00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107860,107861.0,30.0,80.0,0.50,1.97,-0.03,1.94,1.0,0.1,-1.45,...,2.45,0.00,1.94,0.0,NaN,-1.48,0.49,10.016514,2.55,True
107861,107862.0,30.0,80.0,0.55,1.96,-0.03,1.93,1.0,0.1,-1.47,...,2.45,0.00,1.93,0.0,NaN,-1.50,0.46,9.416873,2.55,True
107862,107863.0,30.0,80.0,0.60,1.95,0.00,1.95,0.0,NaN,-1.51,...,2.45,0.00,1.95,0.0,NaN,-1.51,0.44,9.038648,2.45,True
107863,107864.0,30.0,80.0,0.65,1.94,0.00,1.94,0.0,NaN,-1.52,...,2.45,0.00,1.94,0.0,NaN,-1.52,0.42,8.677611,2.45,True


In [28]:
data_filtered.drop(columns=['PMV_vent', 'PMV_cool', 'PMV_heat', 'vent_KW', 'cool_KW', 'heat_KW'], inplace=True)
#data_filtered.drop(columns=['d_PMV_vent', 'd_PMV_cool', 'd_PMV_heat'], inplace=True)

In [35]:
data_filtered.round(2).to_csv(export_filepath + '_output_training_data_filtered.csv')

In [34]:
data_filtered.round(2)

,entry_id,Temperature,Humidity,Air_velocity,PMV,d_PMV_vent,vent_lvl,d_PMV_cool,cool_lvl,d_PMV_heat,heat_lvl,d_PMV_final,PMV_final,PPD_final,KW_final,success
0,1.0,16.0,30.0,0.00,-2.12,0.00,0.0,0.00,0.0,1.90,4.0,1.90,-0.22,6.00,4.00,True
1,2.0,16.0,30.0,0.05,-2.12,0.00,0.0,0.00,0.0,1.90,4.0,1.90,-0.22,6.00,4.00,True
2,3.0,16.0,30.0,0.10,-2.12,0.00,0.0,0.00,0.0,1.85,4.0,1.85,-0.27,6.51,4.00,True
3,4.0,16.0,30.0,0.15,-2.28,0.00,0.0,0.00,0.0,1.90,4.0,1.90,-0.38,8.01,4.00,True
4,5.0,16.0,30.0,0.20,-2.43,0.00,0.0,0.00,0.0,1.97,4.0,1.97,-0.46,9.42,4.00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107860,107861.0,30.0,80.0,0.50,1.97,-0.03,1.0,-1.45,2.0,0.00,0.0,-1.48,0.49,10.02,2.55,True
107861,107862.0,30.0,80.0,0.55,1.96,-0.03,1.0,-1.47,2.0,0.00,0.0,-1.50,0.46,9.42,2.55,True
107862,107863.0,30.0,80.0,0.60,1.95,0.00,0.0,-1.51,2.0,0.00,0.0,-1.51,0.44,9.04,2.45,True
107863,107864.0,30.0,80.0,0.65,1.94,0.00,0.0,-1.52,2.0,0.00,0.0,-1.52,0.42,8.68,2.45,True
